In [1]:
# inspired by tutorial at https://melaniesoek0120.medium.com/stack-overflow-web-scraping-with-python-2a1931ed2bd5

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def href(soup):
    # get all href links from one page 
    href=[]
    for i in soup.find_all("a",class_="s-link",href=True):
        href.append(i['href'])
    return href


In [3]:
def clean_empty_hrefs(hrefs):
   # remove all empty lists
    list_hrefs=[]
    for i in hrefs:
        if i!=[]:
            list_hrefs.append(i)
    # merge all elemenets in one list
    herfs_list=[]
    for i in list_hrefs:
        for j in i:
            herfs_list.append(j)
    return herfs_list


In [4]:
def add_prefix(herfs_list):
    # rearrage those links who do not have 'https://stackoverflow.com' prefix 
    new_href=[]
    prefix='https://stackoverflow.com'
    for h in herfs_list:
        if 'https' not in h:
            m=prefix+h+"answertab=votes#tab-top"
            new_href.append(m)
        else:
            new_href.append(h+"answertab=votes#tab-top")
    return new_href


In [5]:
def single_page_scraper(url):
    req=requests.get(url=url)
    soup=BeautifulSoup(req.text,"html.parser")
    return soup 

def single_page_question_answer(url):
    question=single_page_scraper(url).findAll("a", "question-hyperlink")[0].get_text()
    
    return question


In [6]:
import itertools

from random import randint
from time import sleep

def questions_answers(func_list):
    soups=[]
    for function in func_list:
        full_url = 'https://stackoverflow.com/search?q=%5Br%5D+' + function
        req=requests.get(url=full_url)
        soup=BeautifulSoup(req.text,"html.parser")
        soups.append(soup)
        sleep(randint(2,5))
    
    print("Soups are ready!")
    
    hrefs=[]
    for soup in soups:
        hrefs.append(href(soup)[2:5])

    herfs_list=clean_empty_hrefs(hrefs)

    new_hrefs_list=add_prefix(herfs_list)

    print("All hrefs are ready!")
        
    question_list = []
    
    for url in new_hrefs_list:
        try:
            q=single_page_question_answer(url)
            sleep(randint(2,5))
            question_list.append(q)
        except:
            pass
        
    print("Questions are ready!")
            
    return question_list, new_hrefs_list


In [11]:
functions = ["group_by", "summarise", "mutate", "filter", "rename", "arrange", "select"]

# functions = ["group_by","summarise"]

titles, urls = questions_answers(functions)

Soups are ready!
All hrefs are ready!
Questions are ready!


In [12]:
first_stack = titles[::3]
second_stack = titles[1::3]
third_stack = titles[2::3]

first_links = urls[::3]
second_links = urls[1::3]
third_links = urls[2::3]


In [13]:
df = pd.DataFrame(list(zip(functions, first_stack, second_stack, third_stack, first_links, second_links, third_links)),
                  columns =['function', '1st result', '2nd result', '3rd result', '1st link', '2nd link', '3rd link'])


In [14]:
# df.to_csv("scraper_results.csv")
df


,function,1st result,2nd result,3rd result,1st link,2nd link,3rd link
0,group_by,dplyr::group_by() or just > group_by(),Group_by inside a function,Applying group_by and summarise on data while ...,https://stackoverflow.com/questions/72080570/d...,https://stackoverflow.com/questions/63838280/g...,https://stackoverflow.com/questions/30024437/a...
1,summarise,How to interpret dplyr message `summarise()` r...,Can dplyr summarise over several variables wit...,Dplyr summarise(),https://stackoverflow.com/questions/62140483/h...,https://stackoverflow.com/questions/21295936/c...,https://stackoverflow.com/questions/65851713/d...
2,mutate,Changing factor levels with dplyr mutate,Use dynamic name for new column/variable in `d...,Can dplyr package be used for conditional muta...,https://stackoverflow.com/questions/28190435/c...,https://stackoverflow.com/questions/26003574/u...,https://stackoverflow.com/questions/24459752/c...
3,filter,Filter rows which contain a certain string,Remove rows with all or some NAs (missing valu...,Filter data.frame rows by a logical condition,https://stackoverflow.com/questions/22850026/f...,https://stackoverflow.com/questions/4862178/re...,https://stackoverflow.com/questions/1686569/fi...
4,rename,How to rename a single column in a data.frame?,"Replacement for ""rename"" in dplyr",Rename multiple columns by names,https://stackoverflow.com/questions/7531868/ho...,https://stackoverflow.com/questions/21502465/r...,https://stackoverflow.com/questions/20987295/r...
5,arrange,How do I arrange a variable list of plots usin...,dplyr arrange not arranging by groups,Arranging rows in custom order using dplyr,https://stackoverflow.com/questions/10706753/h...,https://stackoverflow.com/questions/38749417/d...,https://stackoverflow.com/questions/46129322/a...
6,select,"How to join (merge) data frames (inner, outer,...",How to select a CRAN mirror in R,Select first and last row from grouped data,https://stackoverflow.com/questions/1299871/ho...,https://stackoverflow.com/questions/11488174/h...,https://stackoverflow.com/questions/31528981/s...
